In [217]:
"""
The goal is to predict used car prices
"""
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import more_itertools
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import feature_selection
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn import preprocessing

In [218]:
train = pd.read_csv("/Users/trevor/Desktop/Subjects/Kaggle/playground-series-s4e9/used_car_prices/train.csv")
test = pd.read_csv("/Users/trevor/Desktop/Subjects/Kaggle/playground-series-s4e9/used_car_prices/test.csv")

In [219]:
# Look at the columns 
train.head

<bound method NDFrame.head of             id          brand                     model  model_year  milage  \
0            0           MINI             Cooper S Base        2007  213000   
1            1        Lincoln                     LS V8        2002  143250   
2            2      Chevrolet         Silverado 2500 LT        2002  136731   
3            3        Genesis          G90 5.0 Ultimate        2017   19500   
4            4  Mercedes-Benz               Metris Base        2021    7388   
...        ...            ...                       ...         ...     ...   
188528  188528       Cadillac     Escalade ESV Platinum        2017   49000   
188529  188529  Mercedes-Benz  AMG C 43 AMG C 43 4MATIC        2018   28600   
188530  188530  Mercedes-Benz    AMG GLC 63 Base 4MATIC        2021   13650   
188531  188531           Audi          S5 3.0T Prestige        2022   13895   
188532  188532        Porsche                Macan Base        2016   59500   

            fuel_type

In [220]:
cols = train.columns
cols[1:12]

Index(['brand', 'model', 'model_year', 'milage', 'fuel_type', 'engine',
       'transmission', 'ext_col', 'int_col', 'accident', 'clean_title'],
      dtype='object')

In [221]:
# Before we do the pre-processing, let's do some feature engineering!
# The categorical variables actually contain information that would be valuable numerically


train['engine_liters'] = train['engine'].str.extract(r'(\d+\.\d+)L').astype(float)

train['HP'] = train['engine'].str.extract(r'(\d+\.\d+)HP').astype(float)

train['cylinders'] = train['engine'].str.extract(r'(\d+) Cylinder')

train['cylinders'] = train['cylinders'].fillna(0).astype(int)

train['speed'] = train['transmission'].str.extract(r'(\d+)-Speed')

def manual(string):
    if 'Auto' or "A/T" in string:
        return 0
    elif 'Manual' in string:
        return 1
    else:
        return NaN

train['manual'] = train['transmission'].apply(manual)


def accidents(string):
    if isinstance(string, str):
        if 'None' in string:
            return 0 
        else:
            return 1
    

train['accident_bin'] = train['accident'].apply(accidents)

def title(string):
    if isinstance(string, str):
        if "Yes" in string:
            return 1
        else: 
            return 0

train["clean_title_bin"] = train["clean_title"].apply(title)


In [222]:
train["accident_bin"].value_counts()

accident_bin
0.0    144514
1.0     41567
Name: count, dtype: int64

In [223]:
train['fuel_type'].value_counts()

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64

In [224]:
train


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,engine_liters,HP,cylinders,speed,manual,accident_bin,clean_title_bin
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200,1.6,172.0,4,NaN,0,0.0,1.0
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999,3.9,252.0,8,NaN,0,1.0,1.0
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900,5.3,320.0,8,NaN,0,0.0,1.0
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000,5.0,420.0,8,NaN,0,0.0,1.0
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500,2.0,208.0,4,7,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Cadillac,Escalade ESV Platinum,2017,49000,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported,Yes,27500,6.2,420.0,8,NaN,0,0.0,1.0
188529,188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,2018,28600,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported,Yes,30000,3.0,385.0,6,8,0,1.0,1.0
188530,188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,2021,13650,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported,Yes,86900,4.0,469.0,8,7,0,0.0,1.0
188531,188531,Audi,S5 3.0T Prestige,2022,13895,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported,NaN,84900,3.0,NaN,0,1,0,0.0,NaN


In [225]:
print(train['model'].value_counts())
print(train['brand'].value_counts())
# so we have this problem that there are so many models....

model
F-150 XLT                        2945
M3 Base                          2229
Camaro 2SS                       1709
M4 Base                          1622
Mustang GT Premium               1526
                                 ... 
IONIQ Plug-In Hybrid SEL            2
X5 3.0i                             1
X5 xDrive40e                        1
XLR Base                            1
Integra w/A-Spec Tech Package       1
Name: count, Length: 1897, dtype: int64
brand
Ford             23088
Mercedes-Benz    19172
BMW              17028
Chevrolet        16335
Audi             10887
Porsche          10612
Land              9525
Toyota            8850
Lexus             8643
Jeep              6474
Cadillac          4674
RAM               4249
Nissan            3930
Tesla             3738
INFINITI          3276
GMC               3215
Dodge             3133
Mazda             2719
Kia               2497
Lincoln           2423
Subaru            2381
Acura             2282
Honda             2101
H

In [226]:
num = train.iloc[:,1:11].select_dtypes(include='number').columns.tolist()
train[num]

cat = train.iloc[:,1:11].select_dtypes(include='object').columns.tolist()
train[cat]

,brand,model,fuel_type,engine,transmission,ext_col,int_col,accident
0,MINI,Cooper S Base,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported
1,Lincoln,LS V8,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported
2,Chevrolet,Silverado 2500 LT,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported
3,Genesis,G90 5.0 Ultimate,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported
4,Mercedes-Benz,Metris Base,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported
...,...,...,...,...,...,...,...,...
188528,Cadillac,Escalade ESV Platinum,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Beige,None reported
188529,Mercedes-Benz,AMG C 43 AMG C 43 4MATIC,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,At least 1 accident or damage reported
188530,Mercedes-Benz,AMG GLC 63 Base 4MATIC,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,White,Black,None reported
188531,Audi,S5 3.0T Prestige,Gasoline,3.0L,1-Speed Automatic,Daytona Gray Pearl Effect,Black,None reported


In [227]:
train['accident']

0                                  None reported
1         At least 1 accident or damage reported
2                                  None reported
3                                  None reported
4                                  None reported
                           ...                  
188528                             None reported
188529    At least 1 accident or damage reported
188530                             None reported
188531                             None reported
188532                             None reported
Name: accident, Length: 188533, dtype: object

In [228]:
train[['model_year',"milage"]]


,model_year,milage
0,2007,213000
1,2002,143250
2,2002,136731
3,2017,19500
4,2021,7388
...,...,...
188528,2017,49000
188529,2018,28600
188530,2021,13650
188531,2022,13895


In [230]:
minmax = preprocessing.StandardScaler()
train[num] = minmax.fit_transform(train[num])


train2 = train[["model_year", "milage", "engine_liters", "HP", "speed", "manual", "accident_bin", "clean_title_bin", "ext_col", "brand", "price"]]
train2 = pd.get_dummies(train2, columns=['ext_col', 'brand'])
train2

,model_year,milage,engine_liters,HP,speed,manual,accident_bin,clean_title_bin,price,ext_col_Agate Black Metallic,...,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Tesla,brand_Toyota,brand_Volkswagen,brand_Volvo,brand_smart
0,-1.559808,2.957842,1.6,172.0,NaN,0,0.0,1.0,4200,False,...,False,False,False,False,False,False,False,False,False,False
1,-2.443052,1.557184,3.9,252.0,NaN,0,1.0,1.0,4999,False,...,False,False,False,False,False,False,False,False,False,False
2,-2.443052,1.426276,5.3,320.0,NaN,0,0.0,1.0,13900,False,...,False,False,False,False,False,False,False,False,False,False
3,0.206679,-0.927854,5.0,420.0,NaN,0,0.0,1.0,45000,False,...,False,False,False,False,False,False,False,False,False,False
4,0.913274,-1.171076,2.0,208.0,7,0,0.0,1.0,97500,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,0.206679,-0.335461,6.2,420.0,NaN,0,0.0,1.0,27500,False,...,False,False,False,False,False,False,False,False,False,False
188529,0.383328,-0.745116,3.0,385.0,8,0,1.0,1.0,30000,False,...,False,False,False,False,False,False,False,False,False,False
188530,0.913274,-1.045328,4.0,469.0,7,0,0.0,1.0,86900,False,...,False,False,False,False,False,False,False,False,False,False
188531,1.089923,-1.040409,3.0,NaN,1,0,0.0,NaN,84900,False,...,False,False,False,False,False,False,False,False,False,False


,id,model,model_year,milage,fuel_type,engine,transmission,int_col,accident,clean_title,...,brand_Saab,brand_Saturn,brand_Scion,brand_Subaru,brand_Suzuki,brand_Tesla,brand_Toyota,brand_Volkswagen,brand_Volvo,brand_smart
0,0,Cooper S Base,-1.559808,2.957842,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,None reported,Yes,...,False,False,False,False,False,False,False,False,False,False
1,1,LS V8,-2.443052,1.557184,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Beige,At least 1 accident or damage reported,Yes,...,False,False,False,False,False,False,False,False,False,False
2,2,Silverado 2500 LT,-2.443052,1.426276,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Gray,None reported,Yes,...,False,False,False,False,False,False,False,False,False,False
3,3,G90 5.0 Ultimate,0.206679,-0.927854,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,None reported,Yes,...,False,False,False,False,False,False,False,False,False,False
4,4,Metris Base,0.913274,-1.171076,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Beige,None reported,Yes,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188528,188528,Escalade ESV Platinum,0.206679,-0.335461,Gasoline,420.0HP 6.2L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Beige,None reported,Yes,...,False,False,False,False,False,False,False,False,False,False
188529,188529,AMG C 43 AMG C 43 4MATIC,0.383328,-0.745116,Gasoline,385.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,8-Speed A/T,Black,At least 1 accident or damage reported,Yes,...,False,False,False,False,False,False,False,False,False,False
188530,188530,AMG GLC 63 Base 4MATIC,0.913274,-1.045328,Gasoline,469.0HP 4.0L 8 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,None reported,Yes,...,False,False,False,False,False,False,False,False,False,False
188531,188531,S5 3.0T Prestige,1.089923,-1.040409,Gasoline,3.0L,1-Speed Automatic,Black,None reported,NaN,...,False,False,False,False,False,False,False,False,False,False


11